## Chapter 4. Working with Remote Machines

### Connecting to Remote Machines with SSH

To initialize an SSH connection to a host (in this case, biocluster.myuniversity.edu), we use the ssh command:

```bash
$ ssh biocluster.myuniversity.edu
Password:
```
SSH also works with IP addresses—for example, you could connect to a machine with `ssh 192.169.237.42`.

If your server uses a different port than the default (port 22), or your username on the remote machine is different from your local username, you’ll need to specify these details when connecting:

```bash
$ ssh -p 50453 cdarwin@biocluster.myuniversity.edu
```

#### Storing Your Frequent SSH Hosts

SSH config files store details about hosts you frequently connect to. To create a file, just create and edit the file at ~/.ssh/config. Each
entry takes the following form:
    
    Host bio_serv
        HostName 192.168.237.42
        User cdarwin
        Port 50453
        
You won’t need to specify Port and User unless these differ from the remote host’s defaults. With this file saved, you can SSH into 192.168.236.42 using the alias `ssh bio_serv` rather than typing out `ssh -p 50453 cdarwin@192.169.237.42`.

### Quick Authentication with SSH Keys

To avoid retyping remote account password everytime we connect to them, we can use *SSH public key*. To generate public/private SSH keys, we simply run `ssh-keygen`. 
You can distribute your public key to other servers, but your private key must be kept safe and secure and never shared. Let’s generate an SSH key pair using `ssh-keygen`:

```bash
$ ssh-keygen -b 2048
```
This creates a private key at `~/.ssh/id_rsa` and a public key at `~/.ssh/id_rsa.pub`.

Note that the `-b` option stands for "bits", and it specifies the length of the key in bits.
- `-b 2048` means you are generating a 2048-bit key.
- This applies to the type of key you're generating (e.g., RSA by default unless otherwise specified).
- A longer key means stronger security (but slightly more CPU usage).

To use password-less authentication using SSH keys, follow the steps below:
1. SSH to remote host and log in with password
2. Change directories to `~/.ssh/`
3. Append your public key `~/.ssh/id_rsa.pub` to `~/.ssh/authorized_keys` on remote host.

We can skip steps above steps by simply running `ssh-copy-id user@remote_host`.

Once *ssh public key* is copied to remote account, you can log in via `ssh user@remote_host` without a password (unless your key is passphrase-protected). If it is passphrase-protected, we can use `ssh-agent`. The `ssh-agent` program runs in the background on your local machine, and manages your SSH key(s). Then, to tell ssh-agent about our key, we use `ssh-add`.


### Maintaining Long-Running Jobs with nohup and tmux

Processes are terminated if we disconnect from our servers or if our network connection temporarily drops out. To avoid this, we’ll look at two preferable solutions:
`nohup` and `tmux`.

#### nohup

`nohup` is simple command that executes a command and catches hangup signals sent from the terminal. Because the `nohup` command is catching and ignoring these
hangup signals, the program you’re running won’t be interrupted.

```bash
$ nohup program1 > output.txt &
[1] 10900
```
`nohup` returns the process ID number (or PID), which is how you can monitor or terminate this process if you need to.

#### Tmux

`Tmux` (and terminal multiplexers in general) allow you to create a session containing multiple windows, each capable of running their own processes. `Tmux`’s sessions are persistent, meaning that all windows and their processes can easily be restored by reattaching the session. All of `Tmux`’s sessions can be reattached to whatever terminal you’re currently on — just SSH back into the remote host and reattach the Tmux session. All windows will be undisturbed and all processes still running.

Tmux configuration can be updated on `~/.tmux.conf` file. We can simply reload tmux with new configuration by running `tmux source-file ~/.tmux.conf`.

#### Creating, Detaching, and Attaching Tmux Sessions

`Tmux` allows you to have multiple *sessions*, and within each session have multiple windows. Each Tmux session is a separate environment. We can use a session for
each different projects. Unix terminal program’s concept of tabs and windows is entirely different from Tmux’s. Unlike Tmux, your terminal cannot maintain persistent sessions.

To create a new tmux session:

```bash
$tmux new-session -s <session_name>
```
We detach a session with `C-a d` command. We can attach (reattach) a session with `tmux attach-session -t <session_name>`.

In each session, we can create a new window with `C-a c` command. We can move forward or backward through windows with `C-a n` and `C-a p` commands, respectively.

See `man tmux` for a complete list, or press `Control-a ?` from within a Tmux session.

To list all tmux sessions, run `tmux list-sessions`.
